# Excel

### List files

In [2]:
import os
import glob

folder_path = "/mnt/d/Techcombank_/chatbot_document/data/data_real"

# List all Excel files in the folder
excel_files = glob.glob(os.path.join(folder_path, "*.xlsx")) + glob.glob(os.path.join(folder_path, "*.xls"))
print(f"Total Excel files found: {len(excel_files)}")
print(f"Excel files found in {folder_path}:")
for file in excel_files:
    print(os.path.basename(file))

Total Excel files found: 8
Excel files found in /mnt/d/Techcombank_/chatbot_document/data/data_real:
Bao_cao_tai_chinh_tong_hop-1-1.xlsx
BCTD-4.xlsx
call-rp.xlsx
Cong-cu-danh-gia-RR-MTXH-_ES-Tool_Biz-CAG.xlsx
dien-giai-131_154-cham-luan-chuyen.xlsx
phan-tich-ton-kho-phai-thu-phai-tra.xlsx
tong-hop.xlsx
~$BCTD-4.xlsx


### Parsing

In [10]:
from typing import List
import pandas as pd
from langchain.schema import Document
from pathlib import Path
import json

def parse_excel(file_path: Path) -> List[Document]:
    """
    Parse Excel file một cách tối ưu, biến mỗi hàng của mỗi bảng thành một Document riêng biệt
    với metadata chi tiết.
    """
    documents = []
    
    try:
        # Sử dụng ExcelFile để có thể truy cập các sheet hiệu quả
        excel_file = pd.ExcelFile(file_path)
        
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            
            # --- Xử lý cho sheet rỗng ---
            if df.empty:
                continue
                
            # --- Logic để xác định các bảng riêng biệt trong một sheet ---
            df['is_empty'] = df.isnull().all(axis=1)
            df['table_id'] = df['is_empty'].cumsum()
            
            tables = df.groupby('table_id')
            
            for table_id, table_df in tables:
                # Bỏ các dòng trống đã dùng làm dấu phân cách
                table_df = table_df.dropna(how='all').reset_index(drop=True)
                table_df = table_df.drop(columns=['is_empty', 'table_id'], errors='ignore')

                if table_df.empty:
                    continue
                
                headers = table_df.columns.tolist()

                for index, row in table_df.iterrows():
                    row_texts = [
                        f"{str(col_name).strip()}: {str(row[col_name]).strip()}"
                        for col_name in headers if pd.notna(row[col_name])
                    ]
                    
                    if not row_texts:
                        continue
                        
                    page_content = " | ".join(row_texts)
                    
                    metadata = {
                        "source": str(file_path),
                        "file_type": "excel",
                        "sheet_name": sheet_name,
                        "table_id": f"table_{table_id}",
                        "row_index_in_table": index
                    }
                    
                    documents.append(Document(
                        page_content=page_content,
                        metadata=metadata
                    ))

    except Exception as e:
        print(f"Error parsing Excel file (optimized) {file_path}: {e}")
            
    return documents

# Parse all Excel files in the folder
if __name__ == "__main__":
    all_documents = []
    for file_path in excel_files:
        file_path = Path(file_path)
        print(f"Parsing file: {file_path}")
        documents = parse_excel(file_path)
        all_documents.extend(documents)
        #Save temporary results to avoid losing progress
        print(f"Parsed {len(documents)} documents from {file_path}")
    # Save all documents to JSON file with descriptive name
    output_file = Path("/mnt/d/Techcombank_/chatbot_document/data/output/excel_documents_parsed.json")
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w", encoding="utf-8") as f:
        json.dump([{"page_content": doc.page_content, "metadata": doc.metadata} for doc in all_documents], f, ensure_ascii=False, indent=4)
    print(f"Total documents created: {len(all_documents)}")



Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/Bao_cao_tai_chinh_tong_hop-1-1.xlsx
Parsed 57 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/Bao_cao_tai_chinh_tong_hop-1-1.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/BCTD-4.xlsx
Error parsing Excel file (optimized) /mnt/d/Techcombank_/chatbot_document/data/data_real/BCTD-4.xlsx: [Errno 2] No such file or directory: '/mnt/d/Techcombank_/chatbot_document/data/data_real/BCTD-4.xlsx'
Parsed 0 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/BCTD-4.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/call-rp.xlsx
Parsed 104 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/call-rp.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/Cong-cu-danh-gia-RR-MTXH-_ES-Tool_Biz-CAG.xlsx


/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and w

Parsed 785 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/Cong-cu-danh-gia-RR-MTXH-_ES-Tool_Biz-CAG.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/dien-giai-131_154-cham-luan-chuyen.xlsx
Parsed 115 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/dien-giai-131_154-cham-luan-chuyen.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/phan-tich-ton-kho-phai-thu-phai-tra.xlsx
Parsed 235 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/phan-tich-ton-kho-phai-thu-phai-tra.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/tong-hop.xlsx
Parsed 13 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/tong-hop.xlsx
Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/~$BCTD-4.xlsx
Error parsing Excel file (optimized) /mnt/d/Techcombank_/chatbot_document/data/data_real/~$BCTD-4.xlsx: [Errno 2] No such file or directory: '/mnt/d/Techcombank_/chatbot_docu

/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/mnt/d/Techcombank_/chatbot_document/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# PDF

### List files

In [3]:
# List all PDF files in the folder
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))
print(f"Total PDF files found: {len(pdf_files)}")
print(f"PDF files found in {folder_path}:")
for file in pdf_files:
    print(os.path.basename(file))

Total PDF files found: 0
PDF files found in /mnt/d/Techcombank_/chatbot_document/data/data_real:


### Parsing

In [18]:
import os
import pandas as pd
import fitz  # PyMuPDF - cần cho pdf2image
import pdfplumber
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from pathlib import Path
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
def parse_pdf(file_path: Path) -> List[Document]:
        """
        Optimized PDF parser: extracts tables row-by-row and text paragraph-by-paragraph.
        Includes OCR fallback.
        """
        
        # Create a text splitter for chunking large texts
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        documents = []
        
        # --- Method 1: Direct text and table extraction ---
        try:
            with pdfplumber.open(file_path) as pdf:
                for page_num, page in enumerate(pdf.pages, start=1):
                    # 1. Extract tables first
                    tables = page.extract_tables()
                    for table_idx, table in enumerate(tables):
                        # Giả định hàng đầu tiên là header
                        headers = [h.strip() if h else f"col_{i}" for i, h in enumerate(table[0])]
                        for row_idx, row in enumerate(table[1:], start=1):
                            row_texts = [
                                f"{headers[i]}: {str(cell).strip()}"
                                for i, cell in enumerate(row) if cell and str(cell).strip()
                            ]
                            if not row_texts: continue
                            
                            page_content = " | ".join(row_texts)
                            metadata = {
                                "source": str(file_path), "file_type": "pdf", "page_num": page_num,
                                "content_type": "table_row", "table_id": table_idx, 
                                "row_index_in_table": row_idx, "extraction_method": "text"
                            }
                            documents.append(Document(page_content=page_content, metadata=metadata))

                    # 2. Extract text and chunk it
                    page_text = page.extract_text()
                    if page_text and page_text.strip():
                        text_chunks = text_splitter.split_text(page_text)
                        for chunk in text_chunks:
                            metadata = {
                                "source": str(file_path), "file_type": "pdf", "page_num": page_num,
                                "content_type": "paragraph", "extraction_method": "text"
                            }
                            documents.append(Document(page_content=chunk, metadata=metadata))
                            
        except Exception as e:
            print(f"Error during direct PDF parsing for {file_path}: {e}")

        # --- Method 2: OCR Fallback if direct extraction yields little content ---
        if not documents or len("".join(d.page_content for d in documents)) < 100:
            print(f"PDF {file_path} has little text, trying OCR...")
            try:
                images = convert_from_path(file_path, dpi=300)
                for page_num, image in enumerate(images, start=1):
                    ocr_text = pytesseract.image_to_string(image, lang='vie+eng')
                    if ocr_text and ocr_text.strip():
                        text_chunks = text_splitter.split_text(ocr_text)
                        for chunk in text_chunks:
                            metadata = {
                                "source": str(file_path), "file_type": "pdf", "page_num": page_num,
                                "content_type": "paragraph", "extraction_method": "ocr"
                            }
                            documents.append(Document(page_content=chunk, metadata=metadata))
            except Exception as e:
                print(f"Error during OCR PDF parsing for {file_path}: {e}")

        return documents
    
# Parse all PDF files in the folder
if __name__ == "__main__":
    all_documents = []
    for file_path in pdf_files:
        file_path = Path(file_path)
        print(f"Parsing file: {file_path}")
        documents = parse_pdf(file_path)
        all_documents.extend(documents)
        #Save temporary results to avoid losing progress
        print(f"Parsed {len(documents)} documents from {file_path}")
    # Save all documents to JSON file with descriptive name
    output_file = Path("/mnt/d/Techcombank_/chatbot_document/data/output/pdf_documents_parsed.json")
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w", encoding="utf-8") as f:
        json.dump([{"page_content": doc.page_content, "metadata": doc.metadata} for doc in all_documents], f, ensure_ascii=False, indent=4)
    print(f"Total documents created: {len(all_documents)}")

Total documents created: 0


# Word

### List files

In [19]:
# List all Word files in the folder
word_files = glob.glob(os.path.join(folder_path, "*.docx")) + glob.glob(os.path.join(folder_path, "*.doc"))
print(f"Total Word files found: {len(word_files)}")
print(f"Word files found in {folder_path}:")
for file in word_files:
    print(os.path.basename(file))

Total Word files found: 1
Word files found in /mnt/d/Techcombank_/chatbot_document/data/data_real:
SO-SÁNH-DN-CÙNG-NGÀNH.docx


### Parsing

In [20]:
import os
import pandas as pd
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from pathlib import Path
from typing import List
import docx
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
def parse_word(file_path: Path) -> List[Document]:
    """
    Optimized Word parser: extracts tables row-by-row and text paragraph-by-paragraph.
    """
    # Create a text splitter for chunking large texts
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    
    documents = []
    try:
        doc = docx.Document(file_path)

        # 1. Extract tables first
        for table_idx, table in enumerate(doc.tables):
            if not table.rows:
                continue
            
            # Assume the first row is the header
            headers = [cell.text.strip() for cell in table.rows[0].cells]
            
            # Iterate over data rows
            for row_idx, row in enumerate(table.rows[1:], start=1):
                row_texts = []
                for i, cell in enumerate(row.cells):
                    cell_text = cell.text.strip()
                    if cell_text:
                        # Use header if available, otherwise use column index
                        header = headers[i] if i < len(headers) else f"col_{i}"
                        row_texts.append(f"{header}: {cell_text}")
                
                if not row_texts: continue

                page_content = " | ".join(row_texts)
                metadata = {
                    "source": str(file_path),
                    "file_type": "word",
                    "content_type": "table_row",
                    "table_id": table_idx,
                    "row_index_in_table": row_idx
                }
                documents.append(Document(page_content=page_content, metadata=metadata))
        
        # 2. Extract and chunk paragraph text
        # The doc.paragraphs object intelligently excludes text within tables.
        full_text = "\n\n".join(
            para.text.strip() for para in doc.paragraphs if para.text.strip()
        )
        
        if full_text:
            text_chunks = text_splitter.split_text(full_text)
            for chunk in text_chunks:
                metadata = {
                    "source": str(file_path),
                    "file_type": "word",
                    "content_type": "paragraph"
                }
                documents.append(Document(page_content=chunk, metadata=metadata))

    except Exception as e:
        print(f"Error parsing Word file (optimized) {file_path}: {e}")

    return documents

# Parse all Word files in the folder
if __name__ == "__main__":
    all_documents = []
    for file_path in word_files:
        file_path = Path(file_path)
        print(f"Parsing file: {file_path}")
        documents = parse_word(file_path)
        all_documents.extend(documents)
        #Save temporary results to avoid losing progress
        print(f"Parsed {len(documents)} documents from {file_path}")
    # Save all documents to JSON file with descriptive name
    output_file = Path("/mnt/d/Techcombank_/chatbot_document/data/output/word_documents_parsed.json")
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w", encoding="utf-8") as f:
        json.dump([{"page_content": doc.page_content, "metadata": doc.metadata} for doc in all_documents], f, ensure_ascii=False, indent=4)
    print(f"Total documents created: {len(all_documents)}")

Parsing file: /mnt/d/Techcombank_/chatbot_document/data/data_real/SO-SÁNH-DN-CÙNG-NGÀNH.docx
Parsed 6 documents from /mnt/d/Techcombank_/chatbot_document/data/data_real/SO-SÁNH-DN-CÙNG-NGÀNH.docx
Total documents created: 6


# Text

#### List files

In [21]:
# List all text files in the folder
text_files = glob.glob(os.path.join(folder_path, "*.txt"))
print(f"Total text files found: {len(text_files)}")
print(f"Text files found in {folder_path}:")
for file in text_files:
    print(os.path.basename(file))

Total text files found: 0
Text files found in /mnt/d/Techcombank_/chatbot_document/data/data_real:


### Parsing

In [22]:
def parse_text(file_path: Path) -> List[Document]:
        """
        Optimized Text parser: reads the entire file and splits it into manageable
        text chunks using the class's text_splitter.
        """
        text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        )
        documents = []
        content = ""
        encoding_used = "utf-8"
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except UnicodeDecodeError:
            print(f"UTF-8 decoding failed for {file_path}. Trying latin-1.")
            encoding_used = "latin-1"
            try:
                with open(file_path, 'r', encoding='latin-1') as f:
                    content = f.read()
            except Exception as e:
                print(f"Error reading text file {file_path} with latin-1: {e}")
                return []
        except Exception as e:
            print(f"Error reading text file {file_path}: {e}")
            return []

        if content.strip():
            text_chunks = text_splitter.split_text(content)
            for i, chunk in enumerate(text_chunks):
                metadata = {
                    "source": str(file_path),
                    "file_type": "text",
                    "content_type": "text_chunk",
                    "chunk_id": i,
                    "encoding": encoding_used
                }
                documents.append(Document(page_content=chunk, metadata=metadata))
        
        return documents
    
# Parse all text files in the folder
if __name__ == "__main__":
    all_documents = []
    for file_path in text_files:
        file_path = Path(file_path)
        print(f"Parsing file: {file_path}")
        documents = parse_text(file_path)
        all_documents.extend(documents)
        #Save temporary results to avoid losing progress
        print(f"Parsed {len(documents)} documents from {file_path}")
    # Save all documents to JSON file with descriptive name
    output_file = Path("/mnt/d/Techcombank_/chatbot_document/data/output/text_documents_parsed.json")
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w", encoding="utf-8") as f:
        json.dump([{"page_content": doc.page_content, "metadata": doc.metadata} for doc in all_documents], f, ensure_ascii=False, indent=4)
    print(f"Total documents created: {len(all_documents)}")

Total documents created: 0
